In [ ]:
using Pkg
Pkg.activate(".")
Pkg.status()

In [ ]:
using ModelingToolkit
using DifferentialEquations
#using Interpolations
using PhysicsInformedRegression

# Lotka-Volterra model for predator-prey dynamics
The Lotka-Volterra equations describe the dynamics of predator-prey interactions in an ecosystem. The model consists of two differential equations that represent the growth of prey and the growth of predators based on the availability of prey.

$$
\begin{align*}
\frac{dx}{dt} &= \alpha x - \beta xy \\
\frac{dy}{dt} &= \delta xy - \gamma y
\end{align*}
$$

Where: 
- $x$ is the prey population (e.g., rabbits)
- $y$ is the predator population (e.g., foxes)
- $\alpha$ is the growth rate of prey
- $\beta$ is the rate of predation (how many prey are consumed by predators)
- $\delta$ is the growth rate of predators per prey consumed
- $\gamma$ is the natural death rate of predators


## Model setup in Julia
In julia we choose to use the ([DifferentialEquations.jl](https://diffeq.sciml.ai/stable/) package) to represent the Lotka-Volterra model. The following defined the symbolic parameters and the system of equations.

In [ ]:

## LOTKA VOLTERA
@parameters α β γ δ
@variables t x(t) y(t)
D = Differential(t)
eqs = [D(x) ~ α*x - β*x*y,
    D(y) ~ -γ*y + δ*x*y]


# Define the system
@named sys = ODESystem(eqs, t)
sys = complete(sys)

# Simulation setup
We choose the following parameters for the simulation:

* $\alpha = 1.5$  # Growth rate of prey
* $\beta = 1.0$  # Rate of predation
* $\delta = 3.0$  # Growth rate of predators per prey consumed
* $\gamma = 1.0$  # Natural death rate of predators

aswell as the initial conditions:
* $x_0 = 1,0$  # Initial prey population
* $y_0 = 0.5$  # Initial predator population

And simulate the system over a time span of $t = 0$ to $t = 10$ with $1000$ uniform time steps.


In [ ]:
# Define the initial conditions and parameters
u0 = [x => 1.0,
    y => 1.0]

p = [α => 1.5,
    β => 1.0,
    γ => 3.0,
    δ => 1.0]

# Define the time span
start = 0; stop = 10; len = 1000 
timesteps = collect(range(start, stop, length = len))

# Simulate the system
prob = ODEProblem(sys, u0,(timesteps[1], timesteps[end]) ,p, saveat = timesteps)
sol = solve(prob)

# Compute the derivatives
du_approx =  PhysicsInformedRegression.finite_diff(sol.u, sol.t)


In [ ]:
paramsest = PhysicsInformedRegression.physics_informed_regression(sys, sol.u, du_approx)

#compare the estimated parameters to the true parameters
parameterdict = Dict(p)
for (i, param) in enumerate(parameters(sys))
    println("Parameter $(param) = $(parameterdict[param]) estimated as $(paramsest[param])")
end

In [ ]:
# Plot the results
using Plots
estimated_sol = solve(ODEProblem(sys, u0,(start, stop) ,paramsest), Tsit5(), saveat = timesteps)
plot(sol, label = ["x" "y"], title = "Lotka Volterra", lw = 2, dpi = 600)
plot!(estimated_sol, label = ["x_est" "y_est"], lw = 2, ls = :dash, dpi = 600)